In [1]:
# !python -m pip install ipyturtle3 --quiet
# !pip install mobilechelonian --quiet


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# !pip install svg_turtle --quiet


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import ipyturtle3 as turtle
from svg_turtle import SvgTurtle

SCALE_UP = 3

HEIGHT = 1200 * SCALE_UP
WIDTH = 1200 * SCALE_UP

t = SvgTurtle(WIDTH, HEIGHT)

t.speed(10)

# t.shape("turtle")
t.fillcolor("white")

# base = 10
def decimal_expansion(numerator, denominator):
    """
    A generator function that yields the digits of the decimal expansion
    of the fraction represented by numerator/denominator.
    """
    # Handle the integer part
    integer_part = numerator // denominator
    yield integer_part
    numerator %= denominator

    # Handle the fractional part
    while numerator != 0:
        numerator *= base
        digit = numerator // denominator
        yield digit
        numerator %= denominator

# num = 1
# denom = 169
# base= 10
# ls = [20, 50]

# num = 5
# num = 7
# denom = 1948
# base= 10
# ls = [10, 20, 15, 5, 20, 15]
# ls = [10, 30, 9]


num = 1
denom = 1948
base= 10
ls = [r * SCALE_UP for r in [6, 14, 12]]
# ls = [10]

# num = 1
# denom=23
# base = 12
# ls = [20, 15, 20]
# ls = [20]

# Example usage
fraction = decimal_expansion(num, denom)
# fraction_l = decimal_expansion(num_l, denom_1)
colors = ['red', 'purple', 'blue', 'green', 'orange', 'yellow']

# Remove few numbers
for _ in range(1_000):
    next(fraction)

for x in range(10_000):  # Get first 10 digits of the decimal expansion
    next_dig = next(fraction)
    next_frac = 360 * (next_dig / base)
    # with(hold_canvas(myCanvas)):
        # t.pencolor(colors[x%6])
    # t.pencolor(colors[0])
    t.pencolor((0, 0, 1))
    l = ls[x % len(ls)]
    # t.width(x//100 + 1)
    t.width(0.5)
    t.forward(l)
    if next_frac < 180:
        t.left(next_frac)
    else:
        t.right(360 - next_frac)

t.save_as('example.svg')


In [4]:
!rsvg-convert --background-color white -w {WIDTH} -h {HEIGHT} example.svg -o example.png

In [5]:
BORDER_SIZE = 20
!convert example.png -bordercolor UniqueColor -trim +repage -bordercolor white -border {BORDER_SIZE}x{BORDER_SIZE} output.png

convert-im6.q16: unrecognized color `UniqueColor' @ warning/color.c/GetColorCompliance/1057.


In [6]:
!inkscape -o eps-output.eps example.svg 

## Looking at the Graph

We want to get the maximum degree of a line segment in the graph. So, then we can have some idea of how connected the graph is. Then, we want to use this information to determine the probability that removing a random fraction of line segments would remove a cycle.

In [7]:
# !pip install svgpathtools svgwrite --quiet


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [8]:
from svgpathtools import svg2paths
import numpy as np

def parse_svg(file_path):
    paths, _ = svg2paths(file_path)
    line_segments = []

    # Extract line segments from paths
    for path in paths:
        for line in path:
            line_segments.append((line.start, line.end))

    # TODO: Process overlapping lines here

    return line_segments

# Replace 'your_svg_file.svg' with your SVG file's path
line_segments = parse_svg('example.svg')
print(line_segments), f"{len(line_segments)} line segments"

[((2018.8951216287462+1997.5277005625048j), (2000.8951216287462+1997.5277005625048j)), ((2000.8951216287462+1997.5277005625048j), (1966.9164078649985+2022.2146811587886j)), ((1966.9164078649985+2022.2146811587886j), (1955.7917960675004+1987.9766465721632j)), ((1955.7917960675004+1987.9766465721632j), (1970.3541019662496+1977.3965120308987j)), ((1970.3541019662496+1977.3965120308987j), (1957.3753882025017+2017.3408857152951j)), ((1957.3753882025017+2017.3408857152951j), (1986.4999999999998+1996.180616632766j)), ((1986.4999999999998+1996.180616632766j), (1968.4999999999998+1996.180616632766j)), ((1968.4999999999998+1996.180616632766j), (1981.4787137637477+1956.2362429483696j)), ((1981.4787137637477+1956.2362429483696j), (1992.6033255612458+1990.4742775349953j)), ((1992.6033255612458+1990.4742775349953j), (2007.1656314599948+1979.8941429937308j)), ((2007.1656314599948+1979.8941429937308j), (2041.1443452237424+2004.5811235900148j)), ((2041.1443452237424+2004.5811235900148j), (2005.14434522

(None, '10000 line segments')

In [ ]:
line_segments[0][0]

(2018.8951216287462+1997.5277005625048j)

In [20]:
import svgwrite
import random

def create_randomly_colored_svg(line_segments, p, output_file, size=((WIDTH, HEIGHT))):
    new_segments = []
    # Create a new SVG drawing
    dwg = svgwrite.Drawing(output_file, profile='tiny', size=size)

    # Randomly assign colors to each line based on the probability p
    for segment in line_segments:
        # print(segment)
        start, end = segment
        color = 'rgb(0, 0, 255)' if random.random() < 1 - p else 'rgb(255, 0, 0)'
        new_segments.append((start, end, color))

        dwg.add(dwg.line(start=(start.real, start.imag), end=(end.real, end.imag), stroke=color))

    # Save the SVG file
    dwg.save()
    return new_segments

# Assuming line_segments is the list of line segments
p = 0.13  # Adjust this fraction as needed
new_segments = create_randomly_colored_svg(line_segments, p, 'output_random_colored_lines.svg')


### Check if the graph above has any Red-cycles (pieces that will fall out)

In [21]:
import networkx as nx

def are_close(point1, point2, epsilon):
    return np.linalg.norm(np.array(point1) - np.array(point2)) < epsilon

def add_or_get_vertex(graph, point, epsilon):
    for node in graph.nodes:
        if are_close(node, point, epsilon):
            return node
    graph.add_node(point)
    return point

# This is an n^2 algorithm. You can use a spatial index to make it faster?
def build_graph(line_segments, epsilon):
    G = nx.Graph()
    for i, segment in enumerate(line_segments):
        start, end, color = segment
        start_vertex = add_or_get_vertex(G, start, epsilon)
        end_vertex = add_or_get_vertex(G, end, epsilon)
        G.add_edge(start_vertex, end_vertex, color=color)
        if i % 1000 == 0:
          print(f"Processed {i} line segments")
    return G

# Assuming line_segments is the list of line segments you obtained earlier
epsilon = 50  # Adjust epsilon based on your requirement
graph = build_graph(new_segments, epsilon)
print("Done building graph")

Processed 0 line segments
Processed 1000 line segments
Processed 2000 line segments
Processed 3000 line segments
Processed 4000 line segments
Processed 5000 line segments
Processed 6000 line segments
Processed 7000 line segments
Processed 8000 line segments
Processed 9000 line segments
Done building graph


In [22]:
# Assuming 'graph' is your existing graph and each edge has a color attribute
# Create a subgraph with only red edges
red_edges = [(u, v) for u, v, d in graph.edges(data=True) if d.get('color') == 'rgb(255, 0, 0)']
print(len(red_edges))
red_subgraph = graph.edge_subgraph(red_edges)

# Check for cycles in the red subgraph
try:
    cycle = nx.find_cycle(red_subgraph)
    print("Cycle found:", cycle)
except nx.NetworkXNoCycle:
    print("No cycle found in the red subgraph.")


183
Cycle found: [((1985.226384548736+2165.9022391539847j), (1985.226384548736+2165.9022391539847j))]
